In [5]:
import torch
from scipy.io import wavfile
import torch.utils.data as Data
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [6]:
sample_rate_original,waveform_original= wavfile.read('cello_1_min.wav')
_,waveform_1  = wavfile.read('cello_1_min_delay_1.wav')
_,waveform_2 = wavfile.read('cello_1_min_delay_2.wav')
_,waveform_3 = wavfile.read('cello_1_min_long_delayed.wav')
_,waveform_4 = wavfile.read('cello_1_min_noise_mid.wav')
_,waveform_5 = wavfile.read('cello_1_min_noise_start.wav')
_,test_audio = wavfile.read('test.wav') 
waveform_original = waveform_original.astype('float32')
waveform_original /= 255
print("Shape of waveform: {}".format(np.shape(waveform_original)))
print("Sample rate of waveform: {}".format(sample_rate_original))
print(np.shape(waveform_1))

Shape of waveform: (2867795, 2)
Sample rate of waveform: 44100
(2867795, 2)


C:\Users\darshan19\anaconda3\lib\site-packages\ipykernel_launcher.py:1: WavFileWarning: Chunk (non-data) not understood, skipping it.
  """Entry point for launching an IPython kernel.
C:\Users\darshan19\anaconda3\lib\site-packages\ipykernel_launcher.py:2: WavFileWarning: Chunk (non-data) not understood, skipping it.
  
C:\Users\darshan19\anaconda3\lib\site-packages\ipykernel_launcher.py:3: WavFileWarning: Chunk (non-data) not understood, skipping it.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\darshan19\anaconda3\lib\site-packages\ipykernel_launcher.py:4: WavFileWarning: Chunk (non-data) not understood, skipping it.
  after removing the cwd from sys.path.
C:\Users\darshan19\anaconda3\lib\site-packages\ipykernel_launcher.py:5: WavFileWarning: Chunk (non-data) not understood, skipping it.
  """
C:\Users\darshan19\anaconda3\lib\site-packages\ipykernel_launcher.py:6: WavFileWarning: Chunk (non-data) not understood, skipping it.
  
C:\Users\dar

In [7]:
samples=100
BATCH_SIZE = 20
waveform_1 = waveform_3.astype('float32')
waveform_1 /= 255
waveform_2 = waveform_3.astype('float32')
waveform_2 /= 255
waveform_3 = waveform_3.astype('float32')
waveform_3 /= 255
waveform_4 = waveform_3.astype('float32')
waveform_4 /= 255
waveform_5 = waveform_3.astype('float32')
waveform_5 /= 255
test_audio = test_audio.astype('float32')
test_audio /=255

waveform_original = np.repeat(torch.tensor(np.transpose(waveform_original))[np.newaxis,...], BATCH_SIZE, axis=0)
y1 = np.repeat(torch.tensor(np.transpose(waveform_1))[np.newaxis,...], samples, axis=0)
y2 = np.repeat(torch.tensor(np.transpose(waveform_2))[np.newaxis,...], samples, axis=0)
y3 = np.repeat(torch.tensor(np.transpose(waveform_3))[np.newaxis,...], samples, axis=0)
y4 = np.repeat(torch.tensor(np.transpose(waveform_4))[np.newaxis,...], samples, axis=0)
y5 = np.repeat(torch.tensor(np.transpose(waveform_5))[np.newaxis,...], samples, axis=0)

y = np.concatenate((y1,y2,y3,y4,y5),axis=0)
print(np.shape(waveform_original))


np.random.shuffle(y)



print(np.shape(y3))
print(y3)

torch.Size([20, 2, 2867795])
torch.Size([100, 2, 2867795])
tensor([[[-0.0078,  0.0118, -0.0157,  ..., -3.6078, -3.7608, -3.8902],
         [ 0.0118, -0.0157,  0.0157,  ..., -3.7725, -3.8902, -3.9608]],

        [[-0.0078,  0.0118, -0.0157,  ..., -3.6078, -3.7608, -3.8902],
         [ 0.0118, -0.0157,  0.0157,  ..., -3.7725, -3.8902, -3.9608]],

        [[-0.0078,  0.0118, -0.0157,  ..., -3.6078, -3.7608, -3.8902],
         [ 0.0118, -0.0157,  0.0157,  ..., -3.7725, -3.8902, -3.9608]],

        ...,

        [[-0.0078,  0.0118, -0.0157,  ..., -3.6078, -3.7608, -3.8902],
         [ 0.0118, -0.0157,  0.0157,  ..., -3.7725, -3.8902, -3.9608]],

        [[-0.0078,  0.0118, -0.0157,  ..., -3.6078, -3.7608, -3.8902],
         [ 0.0118, -0.0157,  0.0157,  ..., -3.7725, -3.8902, -3.9608]],

        [[-0.0078,  0.0118, -0.0157,  ..., -3.6078, -3.7608, -3.8902],
         [ 0.0118, -0.0157,  0.0157,  ..., -3.7725, -3.8902, -3.9608]]])


In [8]:
torch.manual_seed(1)

  # size of the minibatch
#z = torch.from_numpy(y)
# load dataset into dataLoader
#print(type(z))
loader = Data.DataLoader(

    dataset=y3,

    batch_size=BATCH_SIZE,

    shuffle=True,

    num_workers=0,

)



class DiscreteNet(torch.nn.Module):
        # use an embedding layer to build a NN

    def __init__(self):

        super(DiscreteNet, self).__init__()

        self.embedding = nn.LSTM(2867795,2)
        self.embedding2 = nn.Linear(2,2867795)


    def forward(self, x):

        y,x = self.embedding(x)
        y = self.embedding2(y)
        return y



f = DiscreteNet()


optimizer = optim.Adam(params=f.parameters(), lr=0.01)
loss = nn.MSELoss()


def run():
    for epoch in range(500):

        for step,(batch_x) in enumerate(loader):
            optimizer.zero_grad()
            fx = f(batch_x)
            print(np.shape(fx),np.shape(waveform_original))
            l = loss(fx,waveform_original)
            l.backward()
            optimizer.step()
            print("Epoch: ",epoch,"Loss: ",l.item())

    print('Finished Training')


run()

torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  0 Loss:  94.85454559326172
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  0 Loss:  94.7296371459961
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  0 Loss:  94.60490417480469
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  0 Loss:  94.4802474975586
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  0 Loss:  94.35600280761719
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  1 Loss:  94.23188018798828
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  1 Loss:  94.10807800292969
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  1 Loss:  93.98442840576172
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  1 Loss:  93.86093139648438
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  1 Loss:  93.73773193359375
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  2 Loss

torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  17 Loss:  84.76637268066406
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  17 Loss:  84.6622314453125
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  18 Loss:  84.5583267211914
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  18 Loss:  84.45451354980469
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  18 Loss:  84.35089111328125
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  18 Loss:  84.24752044677734
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  18 Loss:  84.1443862915039
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  19 Loss:  84.04156494140625
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  19 Loss:  83.93890380859375
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  19 Loss:  83.83638763427734
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch

Epoch:  35 Loss:  76.44855499267578
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  35 Loss:  76.36087036132812
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  35 Loss:  76.2735824584961
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  35 Loss:  76.18658447265625
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  35 Loss:  76.09955596923828
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  36 Loss:  76.01251983642578
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  36 Loss:  75.92562866210938
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  36 Loss:  75.83915710449219
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  36 Loss:  75.75286102294922
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  36 Loss:  75.66658020019531
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  37 Loss:  75.58055114746094
torch.Size([20, 2, 286779

torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  52 Loss:  69.34362030029297
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  52 Loss:  69.26944732666016
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  53 Loss:  69.19525146484375
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  53 Loss:  69.121337890625
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  53 Loss:  69.04756927490234
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  53 Loss:  68.97374725341797
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  53 Loss:  68.9001693725586
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  54 Loss:  68.8268051147461
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  54 Loss:  68.75348663330078
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  54 Loss:  68.6802749633789
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch: 

Epoch:  70 Loss:  63.361976623535156
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  70 Loss:  63.298561096191406
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  70 Loss:  63.23515701293945
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  70 Loss:  63.17189025878906
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  70 Loss:  63.10871887207031
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  71 Loss:  63.04566192626953
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  71 Loss:  62.982688903808594
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  71 Loss:  62.9197998046875
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  71 Loss:  62.85698318481445
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  71 Loss:  62.79439926147461
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  72 Loss:  62.731895446777344
torch.Size([20, 2, 28

...

torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  415 Loss:  25.98557472229004
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  415 Loss:  25.980876922607422
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  415 Loss:  25.976211547851562
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  415 Loss:  25.971513748168945
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  416 Loss:  25.966838836669922
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  416 Loss:  25.962160110473633
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  416 Loss:  25.957502365112305
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  416 Loss:  25.95284652709961
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  416 Loss:  25.948196411132812
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  417 Loss:  25.943557739257812
torch.Size([20, 2, 2867795]) torch.Size([2

torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  432 Loss:  25.605085372924805
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  432 Loss:  25.600929260253906
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  432 Loss:  25.596759796142578
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  433 Loss:  25.59263038635254
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  433 Loss:  25.588468551635742
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  433 Loss:  25.58434295654297
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  433 Loss:  25.580219268798828
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  433 Loss:  25.57610321044922
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  434 Loss:  25.572004318237305
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  434 Loss:  25.567882537841797
torch.Size([20, 2, 2867795]) torch.Size([20

torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  449 Loss:  25.26858901977539
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  449 Loss:  25.264904022216797
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  450 Loss:  25.261213302612305
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  450 Loss:  25.25754165649414
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  450 Loss:  25.253902435302734
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  450 Loss:  25.25025177001953
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  450 Loss:  25.24661636352539
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  451 Loss:  25.242979049682617
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  451 Loss:  25.23933219909668
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  451 Loss:  25.23567771911621
torch.Size([20, 2, 2867795]) torch.Size([20, 2

torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  466 Loss:  24.970800399780273
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  467 Loss:  24.96755027770996
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  467 Loss:  24.96430206298828
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  467 Loss:  24.96105194091797
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  467 Loss:  24.95781135559082
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  467 Loss:  24.95457649230957
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  468 Loss:  24.951339721679688
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  468 Loss:  24.94811248779297
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  468 Loss:  24.94489288330078
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  468 Loss:  24.94169044494629
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 

torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  484 Loss:  24.707399368286133
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  484 Loss:  24.704524993896484
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  484 Loss:  24.701663970947266
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  484 Loss:  24.698772430419922
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  484 Loss:  24.695890426635742
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  485 Loss:  24.693023681640625
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  485 Loss:  24.690187454223633
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  485 Loss:  24.687332153320312
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  485 Loss:  24.684478759765625
torch.Size([20, 2, 2867795]) torch.Size([20, 2, 2867795])
Epoch:  485 Loss:  24.68162727355957
torch.Size([20, 2, 2867795]) torch.Size([

In [10]:
test_audio = torch.tensor(test_audio)
d = f(test_audio.reshape(1,2,2867795))
d = d.reshape(2, 2867795)
wavfile.write('reconstructed_test.wav',sample_rate_original,np.transpose(d.detach().numpy()))



In [16]:
from sklearn.metrics import mean_squared_error 


# Calculation of Mean Squared Error (MSE) 
mean_squared_error(waveform_original[0].detach().numpy(),d.detach().numpy()) 


24.543398